<a href="https://colab.research.google.com/github/MikLay/drone-detection/blob/main/Drone_detect_data_YoLoV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment #2 est YOLOv8
To train YoLov8/n/s/m/l:

* Install YOLOv8 dependencies
* Download custom YOLOv8 object detection data
* Write YOLOv8 Training configuration
* Run YOLOv8 training
* Evaluate YOLOv8 performance
* Compare Precision, Reecall, Inference time
  * Friedman tests
  * Nemenyi test
* Export saved YOLOv8 weights for future inference

#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 25.5 GB RAM, 23.3/166.8 GB disk)


# Download Correctly Formatted Custom Dataset 

We'll download our dataset from Roboflow. Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [2]:
#follow the link below to get your download code from from Roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("uavs-hunter").project("uavs-hunting-simulated")
dataset = project.version(1).download("yolov8")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.2 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=cdd986fd5eb3425291004bd4da922c4de1d0b62104fcecf4aae649cb3567874e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.114, to fix: `pip install ultralytics<=8.0.20`
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format


Extracting Dataset Version Zip to UAVs-hunting-simulated-1 in yolov8:: 100%|██████████| 5034/5034 [00:00<00:00, 8284.32it/s]


In [3]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names:
- '0'
- drone
nc: 2
roboflow:
  license: CC BY 4.0
  project: uavs-hunting-simulated
  url: https://universe.roboflow.com/uavs-hunter/uavs-hunting-simulated/dataset/1
  version: 1
  workspace: uavs-hunter
test: test/images
train: train/images
val: valid/images


## Select YOLOv5 🚀 logger

In [3]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
%pip install -q clearml
import clearml; clearml.browser_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.9 MB/s eta 0:00:00


<IPython.core.display.Javascript object>


🤖 ClearML connected successfully - let's build something! 🚀


In [4]:
# Initialize project
from clearml import Task

# Train Custom YOLOv5 Detector

### Next, we'll fire off trainings!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [8]:
# general configs
img = 640
batch = 16
epochs = 100

### YoLoV8n

In [9]:
task1 = Task.init(project_name='Drone-detect/best-model', task_name='YoloV8n')

ClearML Task: created new task id=a0cdf6685367470fb69e6afb8e69b916


ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/a0cdf6685367470fb69e6afb8e69b916/output/log


In [10]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time

# !python -m torch.distributed.run --nproc_per_node 8 train.py --img 416 --batch-size 128 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights yolov5s.pt --name yolov5s_results_multigpu  --cache --device 0,1,2,3,4,5,6,7
!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml  epochs={epochs} imgsz={img}

100% 6.23M/6.23M [00:00<00:00, 82.4MB/s]
Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/UAVs-hunting-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript

/usr/local/lib/python3.10/dist-packages/nbconvert/filters/widgetsdatatypefilter.py:71: UserWarning:

Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:11<00:00,  2.19it/s]
                   all        802        802      0.826      0.544      0.577       0.27

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      2.32G      1.607       1.62      1.117          5        640: 100% 176/176 [00:49<00:00,  3.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:10<00:00,  2.52it/s]
                   all        802        802      0.728      0.615      0.632      0.313

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/100      2.33G      1.578      1.031      1.138          8        640: 100% 176/176 [00:51<00:00,  3.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:12<00:00,  2.13it/s]
                   all        802        802      0.

In [11]:
task1.close()

### YoLoV8s

In [12]:
task2 = Task.init(project_name='Drone-detect/best-model', task_name='YoloV8n')

ClearML Task: created new task id=f9561c7fe30e47a7ad94f21a29fb0b04
ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/f9561c7fe30e47a7ad94f21a29fb0b04/output/log


In [13]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml  epochs={epochs} imgsz={img}

     75/100      4.02G      1.218     0.5207      1.003          6        640: 100% 176/176 [00:49<00:00,  3.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:11<00:00,  2.25it/s]
                   all        802        802      0.915      0.788      0.859      0.499

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     76/100      4.02G      1.215     0.5344     0.9958          6        640: 100% 176/176 [00:50<00:00,  3.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:10<00:00,  2.43it/s]
                   all        802        802      0.901      0.771      0.851      0.504

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     77/100      4.02G      1.188     0.5077      1.001         11        640: 100% 176/176 [00:50<00:00,  3.51it/s]
                 Class     Images  Instances      Box(P      

In [14]:
task2.close()

### YoLoV8m

In [15]:
task3 = Task.init(project_name='Drone-detect/best-model', task_name='YoloV8m')

ClearML Task: created new task id=0f7e21a4cfce489ab20978a180e80dea
ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/0f7e21a4cfce489ab20978a180e80dea/output/log


In [5]:
!yolo task=detect mode=train model=/content/drive/MyDrive/КМА/Master-Diploma/weights/best-yolov8m.pt data={dataset.location}/data.yaml  epochs={epochs} imgsz={img}

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 398, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/engine/model.py", line 366, in train
    self.trainer = TASK_MAP[self.task][1](overrides=overrides, _callbacks=self.callbacks)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/engine/trainer.py", line 82, in __init__
    self.args = get_cfg(cfg, overrides)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 137, in get_cfg
    raise TypeError(f"'{k}={v}' is of invalid type {type(v).__name__}. "
TypeError: 'epochs={epochs}' is of invalid type str. 'epochs' must be an int (i.e. 'epochs=8')


In [17]:
task3.close()

### YoLoV8l

In [18]:
task4 = Task.init(project_name='Drone-detect/best-model', task_name='YoloV8l')

ClearML Task: created new task id=11ddd29b69c94310969fc4fb0f38700d
ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/11ddd29b69c94310969fc4fb0f38700d/output/log


In [ ]:
!yolo task=detect mode=train model=yolov8l.pt data={dataset.location}/data.yaml  epochs={epochs} imgsz={img}

100% 83.7M/83.7M [00:00<00:00, 122MB/s]
Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/UAVs-hunting-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript,

/usr/local/lib/python3.10/dist-packages/nbconvert/filters/widgetsdatatypefilter.py:71: UserWarning:

Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.



      1/100      10.5G      1.605      1.474      1.146          4        640: 100% 176/176 [02:08<00:00,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:11<00:00,  2.18it/s]
                   all        802        802     0.0379      0.367     0.0177    0.00969
2023-06-06 23:23:29,259 - clearml - INFO - NaN value encountered. Reporting it as '0.0'. Use clearml.Logger.set_reporting_nan_value to assign another value

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      10.8G      1.672     0.9248      1.165          5        640: 100% 176/176 [02:03<00:00,  1.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:12<00:00,  2.16it/s]
                   all        802        802       0.18      0.112     0.0533      0.022

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/100      10.8G

In [ ]:
task4.close()

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
%ls runs/train/yolov5l_results/weights

best.pt  last.pt


In [5]:
task5 = Task.init(project_name='Drone-detect/best-model', task_name='Evaluate-YoloV8n')
!yolo task=detect mode=val model=/content/drive/MyDrive/КМА/Master-Diploma/weights/best-yolov8n.pt data={dataset.location}/data.yaml device=0 conf=0.4 plots=true
task5.close()

ClearML Task: created new task id=338462c9ba844b2b8b460b521f515d88


ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/338462c9ba844b2b8b460b521f515d88/output/log
2023-06-07 08:44:57,073 - clearml.Task - INFO - Storing jupyter notebook directly as code
Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 42.5MB/s]
val: Scanning /content/UAVs-hunting-1/valid/labels... 802 images, 0 backgrounds, 0 corrupt: 100% 802/802 [00:00<00:00, 1649.15it/s]
val: New cache created: /content/UAVs-hunting-1/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  90% 46/51 [00:12<00:01,  3.58it/s]

/usr/local/lib/python3.10/dist-packages/nbconvert/filters/widgetsdatatypefilter.py:71: UserWarning:

Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 51/51 [00:13<00:00,  3.65it/s]
                   all        802        802      0.979      0.698      0.842      0.545
Speed: 0.3ms preprocess, 3.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val


In [6]:
task6 = Task.init(project_name='Drone-detect/best-model', task_name='Evaluate-YoloV8s')
!yolo task=detect mode=val model=/content/drive/MyDrive/КМА/Master-Diploma/weights/best-yolov8s.pt data={dataset.location}/data.yaml device=0 conf=0.4 plots=true
task6.close()

ClearML Task: created new task id=10470a9d3be340dc8bca798cbed78bb5
ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/10470a9d3be340dc8bca798cbed78bb5/output/log
Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients
val: Scanning /content/UAVs-hunting-1/valid/labels.cache... 802 images, 0 backgrounds, 0 corrupt: 100% 802/802 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 51/51 [00:12<00:00,  3.94it/s]
                   all        802        802      0.983      0.706      0.849      0.546


/usr/local/lib/python3.10/dist-packages/nbconvert/filters/widgetsdatatypefilter.py:71: UserWarning:

Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.



Speed: 0.3ms preprocess, 5.4ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/val2


In [9]:
task7 = Task.init(project_name='Drone-detect/best-model', task_name='Evaluate-YoloV8m')
!yolo task=detect mode=val model=/content/drive/MyDrive/КМА/Master-Diploma/weights/best-yolo8m.pt data={dataset.location}/data.yaml device=0 conf=0.4 plots=true
task7.close()

ClearML Task: created new task id=ec77632670d6480c820b528220fce767
ClearML results page: https://app.clear.ml/projects/3447e370f16d49f09a6eec0b62466e5a/experiments/ec77632670d6480c820b528220fce767/output/log
Ultralytics YOLOv8.0.114 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients
val: Scanning /content/UAVs-hunting-1/valid/labels.cache... 802 images, 0 backgrounds, 0 corrupt: 100% 802/802 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 51/51 [00:14<00:00,  3.57it/s]
                   all        802        802      0.966      0.635      0.808      0.534
Speed: 0.3ms preprocess, 11.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val3


/usr/local/lib/python3.10/dist-packages/nbconvert/filters/widgetsdatatypefilter.py:71: UserWarning:

Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.

